### KindScore Modeling

In [13]:
from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from pythainlp.word_vector import thai2vec 
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from pythainlp.tokenize import word_tokenize,sent_tokenize
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.cluster import hierarchy
import dill as pickle

import pythainlp.word_vector
# model = word_vector.get_model()
# model_path = 'thwiki_data/models/'

In [14]:
import matplotlib.pyplot as plt
from scipy.spatial import distance
import numpy.matlib
# import speech_recognition as sr

In [15]:
model_nlp = pythainlp.word_vector.get_model()

### Import NLP for Thai
<p>GitHub : https://github.com/PyThaiNLP/pythainlp</p>
<p>tutorials : https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Word-Vector</p>

In [3]:
from pythainlp import word_tokenize, Tokenizer

In [4]:
# from gensim.models.keyedvectors import Word2VecKeyedVectors
# from gensim.models import KeyedVectors
# from pythainlp.corpus import get_corpus, get_corpus_path

# def _download() -> str:
#     path = get_corpus_path("thai2fit_wv")
#     if not path:
#         download_data("thai2fit_wv")
#         path = get_corpus_path("thai2fit_wv")
#     return path

# def get_model() -> Word2VecKeyedVectors:
#     """
#     Download model
#     :return: `gensim` word2vec model
#     :rtype: gensim.models.keyedvectors.Word2VecKeyedVectors
#     """
#     return KeyedVectors.load_word2vec_format(_download(), binary=True)


# _MODEL = get_model()

In [5]:
def sentence_vectorizer(sentence,dim=300,use_mean=True): # ประกาศฟังก์ชัน sentence_vectorizer
#     model_nlp = get_model()

    s = word_tokenize(sentence)
    vec = np.zeros((1,dim))
    for word in s:
        if word in model_nlp.index2word: 
            vec+= model_nlp.word_vec(word)
        else: pass
    if use_mean: vec /= len(s)
    return vec


In [6]:
def elbow_method(data,k):
    clusters = [KMeans(n_clusters = c,init = 'k-means++').fit(data) for c in k]
    centr_lst = [cc.cluster_centers_ for cc in clusters]
    k_distance = [distance.cdist(data, cent, 'euclidean') for cent in centr_lst]
    clust_indx = [np.argmin(kd,axis=1) for kd in k_distance]
    distances = [np.min(kd,axis=1) for kd in k_distance]
    avg_within = [np.sum(dist)/data.shape[0] for dist in distances]
    return avg_within

In [7]:
def elbow_point(curve):
    n_points = len(curve)
    all_coord = np.vstack((range(n_points), curve)).T
    np.array([range(n_points), curve])
    first_point = all_coord[0]
    line_vec = all_coord[-1] - first_point
    line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))
    vec_from_first = all_coord - first_point
    scalar_product = np.sum(vec_from_first * np.matlib.repmat(line_vec_norm, n_points, 1), axis=1)
    vec_from_first_parallel = np.outer(scalar_product, line_vec_norm)
    vec_to_line = vec_from_first - vec_from_first_parallel
    dist_to_line = np.sqrt(np.sum(vec_to_line ** 2, axis=1))
#     print(dist_to_line)
    idx_best = np.argmax(dist_to_line)
    return idx_best

In [8]:
def KindScoreModel(dataset, featureOne, featureTwo):
    df = dataset
    text = df[featureOne].dropna()
    sentences_newmm= []
    
    for x in text:
        proc_newmm = word_tokenize(x, engine='newmm', keep_whitespace=False)
        lst_newmm = []

        for word in proc_newmm:
            if(word != " "):
                lst_newmm.append(word)
        sentences_newmm.append(lst_newmm)
        
    lst_vector = []
    for i in text:
        lst_vector.append(sentence_vectorizer(i)[0])
    X=np.array(lst_vector)

    wcss = []
    for i in range(1,11):
        kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
        kmeans.fit(X)
        wcss.append(kmeans.inertia_)

    elbow=elbow_point(wcss)

    n = len(wcss)+1
    K= range(1, n)

    print("Optimal Cluster Number: ",K[elbow])

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, wcss, 'bx-')
    ax.plot(K[elbow], wcss[elbow], marker='o', markersize=12,markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
    # ax.set_xticks(np.arange(0,n,1))
    # ax.set_yticks(np.arange(0,max(wcss),0.5))
    plt.grid()
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elow for KMeans clustering')
    plt.show();

    n_clusters = K[elbow]
    clf = KMeans(n_clusters=n_clusters,max_iter=100,init='k-means++',n_init=1)
    labels = clf.fit_predict(X)
    labels
    
    cluster = []
    for index, sentence in enumerate(sentences_newmm):
        cluster.append(labels[index])

    text_cluster = pd.DataFrame({"cluster":cluster})  
    
    df_cluster = df [[featureOne,featureTwo]]
    df_cluster["cluster"] = text_cluster.values
    df_cluster
    return df_cluster,clf

In [9]:
# # Record Audio
# r = sr.Recognizer()
# m = sr.Microphone()
# #set threhold level
# with m as source: r.adjust_for_ambient_noise(source)
# print("Set minimum energy threshold to {}".format(r.energy_threshold))

# # Speech recognition using Google Speech Recognition
# def SpeechRecognition():
# #     file_wav
# #     sr.Microphone()  sr.WavFile(file_wav)
# #     with sr.WavFile("C:/Users/Pack.Apichart/Desktop/re/t1.wav")
#     print("Wake UP!! : said something")
#     with  sr.Microphone()  as source:
#         audio = r.listen(source)
#     try:
#     # for testing purposes, we're just using the default API key
#     # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
#     # instead of `r.recognize_google(audio)`

# # "en-US"
# # "th-TH"
#         print("You said: " + r.recognize_google(audio,language = "th-TH"))
#         return r.recognize_google(audio,language = "th-TH")
#     except sr.UnknownValueError:
#         print("Google Speech Recognition could not understand audio")
#         return "stop now"
#     except sr.RequestError as e:
#         print("Could not request results from Google Speech Recognition service; {0}".format(e))
#         return "Could not request results from Google Speech Recognition service"

# def FileRecognition(file_wav,duration):
#     print("Processing !!")
#     with sr.WavFile(file_wav) as source:
#         audio = r.listen(source,duration)
#     try:
#     # for testing purposes, we're just using the default API key
#     # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
#     # instead of `r.recognize_google(audio)`

# # "en-US"
# # "th-TH"
#         print("You said: " + r.recognize_google(audio,language = "th-TH"))
#         return r.recognize_google(audio,language = "th-TH")
# #         return (r.recognize_google(audio))
#     except sr.UnknownValueError:
#         print("Google Speech Recognition could not understand audio")
#         return "stop now"
#     except sr.RequestError as e:
#         print("Could not request results from Google Speech Recognition service; {0}".format(e))
#         return "Could not request results from Google Speech Recognition service"


Set minimum energy threshold to 48.62553160386514


In [10]:
def sentence_similarity(s1,s2):
    for i in s2:
        cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(i)))
    return cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(s2)))

In [16]:
def Weightscore(path, sentence, dataset, model):
    text = sentence
    cluster = []
    l = []
    
    for word in text:
        l.append(sentence_vectorizer(word)[0])
    KL=np.array(l)
    
    newProblem = model.predict(KL)
    cluster.append(newProblem)
    
    for countgroup in range(len(newProblem)):
        state_score = ""
        state_similarity = 0
        for index in dataset[dataset['cluster'] == newProblem[countgroup]].values:
            check_similarity = sentence_similarity(text[countgroup], index[0])
            if(check_similarity > state_similarity):
                state_similarity = check_similarity
                state_score = index[1]

        new_df = pd.DataFrame(
                    {
                'Vehicle' : [''],
                'restaurant' : [''],
                'food delivery' : [''],
                'Problem' : [text[countgroup]],
                'score' : [state_score],
                'work' : [''],
                'เวลาในการรออาหาร' : [''],
                'ราคาอาหาร' : ['']
            })
        print(text[countgroup], "group: ", newProblem[countgroup], "similarity: " ,state_similarity[0][0], "score :" , state_score)

        new_df.to_csv(path+".csv", mode='a', header=False, index=False)
    

    return pd.read_csv(path+".csv")